In [137]:
from pprint import pprint
import networkx as nx
import pandas
import numpy

with open("edge_head_to_head.csv", "r", encoding="utf-8") as file:
    file = file.readlines()
    matches = [[x.split(",")[1].strip(), x.split(",")[3].strip()] for x in file[1:]]
print(matches)

[['Arsenal', 'Aston Villa'], ['Arsenal', 'Brighton'], ['Arsenal', 'Burnley'], ['Arsenal', 'Chelsea'], ['Arsenal', 'Crystal Palace'], ['Arsenal', 'Everton'], ['Arsenal', 'Fulham'], ['Arsenal', 'Leeds'], ['Arsenal', 'Leicester'], ['Arsenal', 'Liverpool'], ['Arsenal', 'Man City'], ['Arsenal', 'Man Utd'], ['Arsenal', 'Newcastle'], ['Arsenal', 'Sheff Utd'], ['Arsenal', 'Southampton'], ['Arsenal', 'Tottenham'], ['Arsenal', 'West Brom'], ['Arsenal', 'West Ham'], ['Arsenal', 'Wolves'], ['Arsenal', 'Atletico Madrid'], ['Arsenal', 'Real Madrid'], ['Arsenal', 'Barcelona'], ['Arsenal', 'Sevilla'], ['Arsenal', 'Villarreal'], ['Arsenal', 'Celta Vigo'], ['Arsenal', 'Valencia'], ['Arsenal', 'Inter'], ['Arsenal', 'AC Milan'], ['Arsenal', 'Juventus'], ['Arsenal', 'Napoli'], ['Arsenal', 'Lazio'], ['Arsenal', 'Roma'], ['Arsenal', 'Sampdoria'], ['Arsenal', 'Udinese'], ['Arsenal', 'Fiorentina'], ['Arsenal', 'Torino'], ['Arsenal', 'Parma'], ['Arsenal', 'Paris Saint Germain'], ['Arsenal', 'Monaco'], ['Arsenal

In [134]:
# Fetch market values datas
with open("vertex_head_to_head.csv", "r", encoding="utf-8") as file:
    file = file.readlines()
    clubs = [x.split(",")[1].strip() for x in file[1:]]
    col_marketVal = [int(x.split(",")[6]) for x in file[1:]]
    
segments = pandas.qcut(col_marketVal,3)
pprint(segments.categories)
    
col_segment = []
segmented_clubs = {"low":[], "medium":[], "high":[]}
for i in range(len(clubs)):
    if col_marketVal[i] in segments.categories[0]: segmented_clubs["low"].append(clubs[i])
    elif col_marketVal[i] in segments.categories[1]: segmented_clubs["medium"].append(clubs[i])
    elif col_marketVal[i] in segments.categories[2]: segmented_clubs["high"].append(clubs[i])


IntervalIndex([(38699999.999, 129550000.0], (129550000.0, 314800000.0], (314800000.0, 1030000000.0]],
              closed='right',
              dtype='interval[float64]')


In [130]:
# Export segmentation to csv

with open("vertex_segmentation.csv", "w", encoding="utf-8") as filew:
    print("Id,club_name,manager,year_formed,chairman,city,market_value,country,rank,segmentation",file=filew)
    with open("vertex_head_to_head.csv", "r", encoding="utf-8") as file:
        file = file.readlines()
        for line in file:
            club = line.split(",")[1].strip()
            for seg in segmented_clubs:
                if club in segmented_clubs[seg]:
                    print(line.strip()+","+seg,file=filew)

In [108]:
# Creating Graph

net = nx.Graph()

net.add_nodes_from(segmented_clubs["low"], segment="low")
net.add_nodes_from(segmented_clubs["medium"], segment="medium")
net.add_nodes_from(segmented_clubs["high"], segment="high")

net.add_edges_from(matches)

print(nx.attribute_assortativity_coefficient(net, 'segment'))

0.08049762841416785


In [114]:
# Experiment without same-league match

with open("vertex_head_to_head.csv", "r", encoding="utf-8") as file:
    file = file.readlines()
    countries = {x.split(",")[1].strip(): x.split(",")[-2].strip() for x in file[1:]}
    matches_without_diff = matches.copy()
    for match in matches:
        if countries[match[0]] == countries[match[1]]: matches_without_diff.remove(match)

In [115]:
# Fetch market values datas
with open("vertex_head_to_head.csv", "r", encoding="utf-8") as file:
    file = file.readlines()
    clubs = [x for x in countries]
    col_marketVal = []
    for club in clubs:
        for line in file:
            if club in line: col_marketVal.append(int(line.split(",")[6]))
    
segments = pandas.qcut(col_marketVal,3)
    
col_segment = []
segmented_clubs = {"low":[], "medium":[], "high":[]}
for i in range(len(clubs)):
    if col_marketVal[i] in segments.categories[0]: segmented_clubs["low"].append(clubs[i])
    elif col_marketVal[i] in segments.categories[1]: segmented_clubs["medium"].append(clubs[i])
    elif col_marketVal[i] in segments.categories[2]: segmented_clubs["high"].append(clubs[i])

In [138]:
net2 = nx.Graph()

net2.add_nodes_from(segmented_clubs["low"], segment="low")
net2.add_nodes_from(segmented_clubs["medium"], segment="medium")
net2.add_nodes_from(segmented_clubs["high"], segment="high")

net2.add_edges_from(matches_without_diff)

print(nx.attribute_assortativity_coefficient(net2, 'segment'))
print(len(matches))
print(len(matches_without_diff))

0.038637742762073816
1472
559
